In [1]:
from datasets import load_dataset
from fyp_work.paper_to_equation.generation.EquationGenerator import BaseDataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch
import evaluate
import numpy as np

In [ ]:
class T5Dataset(BaseDataset):
    def __init__(self, num, filepath):
        super().__init__(num, filepath)

    def get_columns(self):
        return ["MathML", "Python"]

t5_data = T5Dataset(100, "t5_validation_1.csv")
t5_data.create()

In [2]:
data_files = {"train": "t5_train_1.csv", "validation": "t5_validation_1.csv", "test": "t5_test_1.csv"}
mml_py_dataset = load_dataset("csv", data_files=data_files)

print(mml_py_dataset)
display(mml_py_dataset["train"][0])

DatasetDict({
    train: Dataset({
        features: ['MathML', 'Python'],
        num_rows: 500
    })
    validation: Dataset({
        features: ['MathML', 'Python'],
        num_rows: 100
    })
    test: Dataset({
        features: ['MathML', 'Python'],
        num_rows: 100
    })
})


{'MathML': '\n<mml:msup>\n<mml:msup>\n<mml:mtext>exp</mml:mtext>\n<mml:mi>r</mml:mi>\n</mml:msup>\n<mml:msup>\n<mml:mi>K</mml:mi>\n<mml:msup>\n<mml:mtext>exp</mml:mtext>\n<mml:mrow>\n<mml:mi>ι</mml:mi>\n<mml:mi>ρ</mml:mi>\n</mml:mrow>\n</mml:msup>\n</mml:msup>\n</mml:msup>\n<mml:mo>=</mml:mo>\n<mml:mrow>\n<mml:mi>cos</mml:mi>\n<mml:mfenced>\n<mml:mi>d</mml:mi>\n</mml:mfenced>\n</mml:mrow>\n',
 'Python': "ι = Symbol('ι')\nρ = Symbol('ρ')\nK = Symbol('K')\nr = Symbol('r')\nd = Symbol('d')\ne = Eq(exp(r)**(K**exp(ι*ρ)), cos(d))"}

In [3]:
model_checkpoint = "t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

max_length = 1024
def preprocess_function(examples):
    prefix = "translate: MathML to Python: "
    inputs = [prefix + mml for mml in examples["MathML"]]
    model_inputs = tokenizer(inputs, max_length=max_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["Python"], max_length=max_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = mml_py_dataset.map(preprocess_function, batched=True, remove_columns=["MathML", "Python"])

In [4]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

batch = data_collator([tokenized_dataset["train"][i] for i in range(1,4)])
print(batch["labels"])

tensor([[    3,   172,  3274,     3, 18650,   599,    31,   172,    31,    61,
             3,    17,  3274,     3, 18650,   599,    31,    17,    31,    61,
             3,     4,  3274,     3, 18650,   599,    31,     4,    31,    61,
           454,  3274,     3, 18650,   599,    31,   566,    31,    61,     3,
             2,  3274,     3, 18650,   599,    31,     2,    31,    61,     3,
            15,  3274,   262,  1824,   599,  2152,   599,   172,    61,  1768,
          4303,   599,    17,    61,    87,     4,     6,     3,    17,   152,
           599,     2,    61, 19844,   994,   102,   599,   566,    61,    61,
             1,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  

In [5]:
metric = evaluate.load("sacrebleu")

predictions = ["η = Symbol('η')\nη_0 = Symbol('η_0')\nQ_η = Symbol('Q_η')\nR = Symbol('R')\nT = Symbol('T')\ne = Eq(η, η_0*exp(((Q_η*T)/(R*T)))"]
references = [["η = Symbol('η')\nη_0 = Symbol('η_0')\nQ_η = Symbol('Q_η')\nR = Symbol('R')\nT = Symbol('T')\ne = Eq(η, η_0*exp(Q_η/(R*T)))"]]

metric.compute(predictions=predictions, references=references)

{'score': 88.64759993490114,
 'counts': [61, 59, 56, 53],
 'totals': [66, 65, 64, 63],
 'precisions': [92.42424242424242, 90.76923076923077, 87.5, 84.12698412698413],
 'bp': 1.0,
 'sys_len': 66,
 'ref_len': 61}

In [6]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    # SacreBLEU
    BLEUresult = metric.compute(predictions=decoded_preds, references=decoded_labels)

    # Equation evaluation
    

    return {"bleu": BLEUresult["score"]}   

In [7]:
from huggingface_hub import notebook_login

notebook_login()

In [8]:
args = Seq2SeqTrainingArguments(
    f"t5-base-mathml-to-python",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model, 
    args, 
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# trainer.evaluate(max_length=max_length)

c:\Users\kyanj\anaconda3\envs\fyp_env\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
trainer.train()

  0%|          | 0/48 [00:00<?, ?it/s]

KeyboardInterrupt: 